In [1]:
import tweepy
import configparser
import pandas as pd
import numpy as np
import os
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [21]:
output_path = "//Output//"

In [2]:
#Reading credentials and base files
config = configparser.ConfigParser()
config.read("config.ini")
twitter_handles_df = pd.read_excel("Twitter Handles.xlsx") 

In [ ]:
#Remove all hashtags and the following words
#Remove special characters
#Remove all links

def preProcessTweets(tweet):
    

In [16]:
def getSentiment(sentence):
 
    sia_obj = SentimentIntensityAnalyzer()
 
    sentiment_dict = sia_obj.polarity_scores(sentence)
    
    if sentiment_dict['compound'] >= 0.05 :
        sentiment = "Positive"
 
    elif sentiment_dict['compound'] <= - 0.05 :
        sentiment = "Negative"
 
    else :
        sentiment = "Neutral"
        
    return [sentiment, sentiment_dict['compound']]

In [4]:
###########################################################################################################################
#Function Name:
#Purpose:
#Arguments:
#Output:
###########################################################################################################################

def getAuthenticatedTwitterInstance(config):
    
    api_key = config['Twitter-KYB Config']['api_key']
    api_key_secret = config['Twitter-KYB Config']['api_secret_key']
    access_token = config['Twitter-KYB Config']['access_token']
    access_token_secret = config['Twitter-KYB Config']['access_token_secret']
    
    #Authentication
    auth = tweepy.OAuthHandler(api_key, api_key_secret)
    auth.set_access_token(access_token, access_token_secret)

    twitter_api_instance = tweepy.API(auth)
    
    return twitter_api_instance

In [47]:
###########################################################################################################################
#Function Name:
#Purpose:
#Arguments:
#Output:
###########################################################################################################################

def getBaseTweets(business_df, twitter_api_instance):
    
    base_tweets_df = pd.DataFrame()

    for handle in business_df["Twitter Handles"].tolist():    
        tweets = twitter_api_instance.search_tweets(q = handle + " -filter:retweets", result_type = "recent", count = 2000, tweet_mode = 'extended')
    
        business_type = business_df[business_df["Twitter Handles"] == handle]["Business"].iloc[0]
        brand = business_df[business_df["Twitter Handles"] == handle]["Brands"].iloc[0]
    
        for tweet in tweets:
            tweet_df = pd.DataFrame([{"Business": business_type,
                                      "Brand": brand,
                                      "Tweet": tweet.full_text}])
        
            base_tweets_df = pd.concat([base_tweets_df, tweet_df], axis = 0, ignore_index = True)
    
    return base_tweets_df

In [6]:
twitter_instance = getAuthenticatedTwitterInstance(config)

In [48]:
base_tweets_df = getBaseTweets(twitter_handles_df, twitter_instance)

In [49]:
base_tweets_df["Sentiment"] = base_tweets_df["Tweet"].apply(lambda x: getSentiment(x))

In [50]:
base_tweets_df["Sentiment Score"] = base_tweets_df["Sentiment"].apply(lambda x: x[1])
base_tweets_df["Sentiment"] = base_tweets_df["Sentiment"].apply(lambda x: x[0])

In [51]:
base_tweets_df["Sentiment"].value_counts()

Positive    812
Negative    637
Neutral     519
Name: Sentiment, dtype: int64

In [52]:
base_tweets_df.to_excel("Base Tweet Data Analysis.xlsx", index = False)